# Вариант 12
## Соболев

<div><img src="./Санек пирожок.png" width="450"/></div>

# Функция <em><strong>solve</strong></em> принимает следующие аргументы
##### k - список жесткостей пружин
##### c - список коэффициентов фемпфирования
##### m1,m2,m3 - массы тел
##### h1,h2,h3,h4 - расстояния до верхних пружин от центра масс 1 тела по горизонтали
##### l1,l2,l3,l4 - расстояния до нижних амортизаторов от центра масс 1 тела по горизонтали
##### H1,H2 - расстояния от центров масс до крайних амортизаторов
##### t_matrix - список времен между удароми каждого амортизатора в период времени с (1,3) с шагом 0.002
##### I1,I2,I3 - моменты инерции

### А также имеет следующи константы:
##### delta_t=0.002
##### gamma = 0.5
##### betta = 0.25

In [ ]:
import numpy as np
#Функция для создания синусоиды
def rounding(n):
    return round(round(n,3)+0.001,3) if (round(n,3)*1000)%2!=0 else round(n,3)

def sinys(t,amplisuda,L):
    return amplisuda*np.sin(np.pi*t/L)

def solve(k,c,m1,m2,m3,h1,h2,h3,h4,l1,l2,l3,l4,H1,H2,I1,I2,I3,alfa,v,L,A,delta_t=0.002,gamma = 0.5,betta = 0.25):
    v = v*1000/3600
    T = round((l1+l2+L)/v,2)
    T = [round(i,3) for i in np.arange(0,(T+0.001),delta_t)]
    N = len(T)
    t_matrix = list(map(rounding,[(l2-l2)/v,(l2-l4)/v, (l2+l3)/v,(l2+l1)/v]))
    n_it1t = int((L/v)/delta_t)
    sinus = np.array([sinys(r,A,L) for r in np.linspace(0,1,n_it1t)])
    #Создаем матрицу коэффициентов упругости
    cos_alfa = np.cos(alfa*np.pi/180)
    K_diag = np.diag(k)
    D = np.array([[-1,h1,1,-H1,0,0],
    [-1*cos_alfa,h3*cos_alfa,1*cos_alfa,0,0,0],
    [-1*cos_alfa,(h4-h2)*cos_alfa,0,0,1*cos_alfa,0],
    [-1,-h2,0,0,1,H2],
    [1,-l1,0,0,0,0],
    [1,-l3,0,0,0,0],
    [1,l4,0,0,0,0],
    [1,l2,0,0,0,0]])
    K = np.transpose(D)@K_diag@D
    #Создаем матрицу коэффициентов демпфирования
    C_diag = np.diag(c)
    C = np.array([[0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [1,-l1,0,0,0,0],
    [1,-l3,0,0,0,0],
    [1,l4,0,0,0,0],
    [1,l2,0,0,0,0]])
    C = np.transpose(C)@C_diag@C
    #Создаем диагональную матрицу масс и моментов
    M = np.diag([m1,I1,m2,I2,m3,I3])
    #Делаем матрицу внешних воздействий
    n = D.shape[0]-len(t_matrix)+1
    y_voz = np.zeros((D.shape[0]+1,N))
    y_voz[0,:] = T
    index_t = [np.where(y_voz[0,:] == i)[-1][-1] for i in t_matrix]
    for i in index_t:
        y_voz[n,i:] =  np.concatenate([sinus,np.zeros((N-n_it1t-i))])
        n+=1
    y_impact = y_voz[1:,:]
    #Создаем обратную матрицу
    M_reverse = np.linalg.inv(M+delta_t*gamma*C+delta_t**2*betta*K)
    #Создадим оператор для ввода вектора внешних сил
    K_diag_transpose = np.transpose(D)@K_diag
    #Зададим вектора перемещения скорости и ускорения в 0 момент
    X = np.zeros((K_diag_transpose.shape[0],1))
    X_t = np.zeros((K_diag_transpose.shape[0],1))
    X_tt = np.zeros((K_diag_transpose.shape[0],1))
    for i in range(N-1):
        #Формируем матрицы методом Ньюмарка
        #Ускорений
        F = K_diag_transpose@y_impact[:,i+1]-C@(X_t[:,-1]+delta_t*(1-gamma)*X_tt[:,-1])-K@(X[:,-1]+delta_t*X_t[:,-1]+1/2*delta_t**2*(1-2*betta)*X_tt[:,-1])
        X_tt = np.concatenate([X_tt,(np.dot(F,M_reverse)).reshape(-1,1)],axis =1)
        #Скоростей
        X_tn = X_t[:,i]+delta_t*(gamma*X_tt[:,-1]+(1-gamma)*X_tt[:,i])
        X_t = np.concatenate([X_t,X_tn.reshape(-1,1)],axis =1)
        #Перемещений
        X_n = X[:,i]+delta_t*X_t[:,i]+0.5*(delta_t**2)*(2*betta*X_tt[:,-1]+(1-2*betta)*X_tt[:,i])
        X = np.concatenate([X,X_n.reshape(-1,1)],axis =1)
    return X,X_t,X_tt